In [ ]:
%%javascript
$('#header').toggle()

In [ ]:
import ipywidgets as widgets
import json
import pandas as pd
import qgrid
import requests
from ipyleaflet import FullScreenControl, LayerGroup, Map, Marker
from IPython.display import display, HTML, IFrame
from pyvis.network import Network

In [ ]:
layout_main = {
    'width': '100%',
    'height': '100%',
}

In [ ]:
image_header = widgets.Image(
    value=open('images/DI_NLP_Header.png', 'rb').read(),
    format='png',
    layout=layout_main,
)

In [ ]:
input_url = widgets.Text(
    placeholder='TextCat URL',
    layout=layout_main,
)
input_text = widgets.Textarea(
    placeholder='Text for analysis',
    layout=layout_main,
)

In [ ]:
output_message = widgets.Output()
output_text = widgets.Output()
output_tree = widgets.Output()
output_graph = widgets.Output()
output_map = widgets.Output()
output_table = widgets.Output(
    layout={
        'width': '3000px',
        'height': '100%',
    },
)

In [ ]:
accordion_text = widgets.Accordion(
    children=[output_text],
    layout=layout_main,
)
accordion_text.set_title(0, 'Text')

accordion_tree = widgets.Accordion(
    children=[output_tree],
    layout=layout_main,
)
accordion_tree.set_title(0, 'Tree')

accordion_graph = widgets.Accordion(
    children=[output_graph],
    layout=layout_main,
)
accordion_graph.set_title(0, 'Graph')

accordion_map = widgets.Accordion(
    children=[output_map],
    layout=layout_main,
)
accordion_map.set_title(0, 'Map')

accordion_table = widgets.Accordion(
    children=[output_table],
    layout=layout_main,
)
accordion_table.set_title(0, 'Table')

In [ ]:
@output_message.capture(clear_output=True, wait=True)
def set_output_message(message):
    display(HTML(message))

@output_text.capture(clear_output=True, wait=True)
def set_output_text(html_text):
    accordion_text.selected_index = 0
    display(HTML(html_text))

@output_tree.capture(clear_output=True, wait=True)
def set_output_tree(html_tree):
    accordion_tree.selected_index = 0
    display(HTML(html_tree))

node_counter = 0
@output_graph.capture(clear_output=True, wait=True)
def set_output_graph(ents, tree):
    g_pyvis = Network(
        directed=True,
        notebook=True,
        width='100%',
        height='100%',
    )
    g_pyvis.prep_notebook()
#     g_pyvis.add_nodes(
#         [ent['ent'] for ent in data],
#         color=[ent['color'] for ent in data],
#         title=[
#             pd.DataFrame(
#                 [(key[1], ent[key[0]])
#                  for key in keys_display_order
#                  if key[0] in ent.keys()],
#             ).style.set_table_styles([
#                 {'selector': 'th.col_heading',
#                  'props': [('display', 'none')]},
#                 {'selector': 'th.row_heading',
#                  'props': [('display', 'none')]},
#             ]).render()
#             for ent in data
#         ],
#     )
#     g_pyvis.enable_physics(True)
#     g_pyvis.barnes_hut()
#     g_pyvis.show_buttons(filter_=['nodes'])
#     g_pyvis.show_buttons(filter_=['edges'])
#     g_pyvis.show_buttons(filter_=['physics'])
    def nodes_good(nodes):
# Temporarily allowing all POS while testing:
#         return([node for node in nodes if (node['POS_coarse'] not in {'DET', 'CCONJ', 'PUNCT', 'SYM'})])
        return(nodes)
    def add_nodes(nodes):
        global node_counter
        for node in nodes:
            node.update({'id': node_counter})
            node_counter += 1
            g_pyvis.add_node(
                node['id'],
                label=node['POS_coarse'] + ': ' + node['word'],
                title=node['POS_coarse'] + ': ' + node['word'],
                font='14px gillsans #424242',
                color=ents[node['word']]['color'] if (node['word'] in ents.keys()) else '#EBEBEB',
            )
            sub_nodes = nodes_good(node['modifiers'])
            if sub_nodes:
                add_nodes(sub_nodes)
                for sub_node in sub_nodes:
                    g_pyvis.add_edge(
                        node['id'],
                        sub_node['id'],
                        label=sub_node['arc'],
                        title=sub_node['arc'],
                        font='12px gillsans-italic #A9A9A9',
                    )
    add_nodes(nodes_good(tree))
    node_counter = 0
    g_pyvis.force_atlas_2based(
        gravity=-100,
        central_gravity=0.005,
        spring_length=50,
        spring_strength=0.1,
        damping=0.5,
        overlap=0.5,
    )
    g_pyvis.show('images/DI_NLP_Graph.html')
    accordion_graph.selected_index = 0
    display(
        IFrame(
            'images/DI_NLP_Graph.html',
            width='100%',
            height='1000px',
        ),
    )

@output_map.capture(clear_output=True, wait=True)
def set_output_map(ents):
    m = Map(
        center=(20.0, 0.0),
        zoom=2,
    )
    markers = []
    for ent in ents.keys():
        if  (ents[ent]['ent_type'] == 'LOC') \
        and ('geo_lat' in ents[ent].keys()) \
        and ('geo_lon' in ents[ent].keys()):
            markers.append(Marker(
                location=(ents[ent]['geo_lat'], ents[ent]['geo_lon']),
                title=ent,
                draggable=False,
            ))
    m.add_layer(LayerGroup(
        layers=tuple(markers),
    ))
#     m.add_layer(MarkerCluster(
#         markers=tuple(markers),
#     ))
    m.add_control(FullScreenControl())
    accordion_map.selected_index = 0
    display(m)

@output_table.capture(clear_output=True, wait=True)
def set_output_table(ents, keys_display_order):
    ents = [ents[ent] for ent in ents.keys()]
    ents = sorted(ents, key=lambda ent:ent['tokens_idx'][0])
    df = pd.DataFrame(
        [[ent[key[0]] if key[0] in ent.keys() else ''
          for key in keys_display_order]
         for ent in ents],
        columns=[key[1] for key in keys_display_order],
    )
    qgrid_widget = qgrid.show_grid(
        df,
        grid_options={
            'enableColumnReorder': True,
        },
#         show_toolbar=True, # This gives a full-screen button, but also add/remove row buttons which we don't want ...
    )
    accordion_table.selected_index = 0
    display(qgrid_widget)
#     df = df.style.set_table_styles([
#         {'selector': 'td',
#          'props': [('text-align', 'left')]},
#         {'selector': 'th.col_heading',
#          'props': [('text-align', 'left')]},
#         {'selector': 'th.row_heading',
#          'props': [('text-align', 'right')]},
#     ])
#     display(df)
#     display(TableDisplay(df))

In [ ]:
def toggle(dummy):
    if (accordion_text.selected_index == 0) \
    or (accordion_tree.selected_index == 0) \
    or (accordion_graph.selected_index == 0) \
    or (accordion_map.selected_index == 0) \
    or (accordion_table.selected_index == 0):
        accordion_text.selected_index = None
        accordion_tree.selected_index = None
        accordion_graph.selected_index = None
        accordion_map.selected_index = None
        accordion_table.selected_index = None
    else:
        accordion_text.selected_index = 0
        accordion_tree.selected_index = 0
        accordion_graph.selected_index = 0
        accordion_map.selected_index = 0
        accordion_table.selected_index = 0

def clear(dummy):
    input_text.value=''
    output_message.clear_output()
    output_text.clear_output()
    output_tree.clear_output()
    output_graph.clear_output()
    output_map.clear_output()
    output_table.clear_output()
    accordion_text.selected_index = None
    accordion_tree.selected_index = None
    accordion_graph.selected_index = None
    accordion_map.selected_index = None
    accordion_table.selected_index = None

def textcat(dummy):
    if   (not input_url.value) and (not input_text.value):
        set_output_message('ERROR: Enter TextCat URL and text for analysis')
    elif (not input_url.value):
        set_output_message('ERROR: Enter TextCat URL')
    elif (not input_text.value):
        set_output_message('ERROR: Enter text for analysis')
    else:
        set_output_message('Processing ...')
# TODO: Explicitly specify as unicode input
        r = requests.post(
            input_url.value,
            data=json.dumps({'text': input_text.value}),
            params={'return_app': 'true'},
            headers={'Content-Type': 'application/json'},
        )
        if r.ok and r.json():
            set_output_message('Done in: ' + str(r.json()['timer']) + ' s')
            set_output_table(r.json()['ents'], r.json()['keys_display_order'])
            set_output_map(r.json()['ents'])
            set_output_graph(r.json()['ents'], r.json()['tree'])
            set_output_tree(r.json()['html_tree'])
            set_output_text(r.json()['html_text'])
        else:
            set_output_message('ERROR: No data received')
            clear(None)

In [ ]:
button_toggle = widgets.Button(
    icon='list',
    tooltip='Toggle all output displays',
    layout={
        'width': '50px',
    },
)
button_toggle.on_click(toggle)

button_clear = widgets.Button(
    icon='refresh',
    tooltip='Clear input and output',
    layout={
        'width': '50px',
    },
)
button_clear.on_click(clear)

button_go = widgets.Button(
    icon='play',
    tooltip='Go!',
    layout={
        'width': '50px',
    },
)
button_go.on_click(textcat)

In [ ]:
VBox_main = widgets.VBox(
    children=[
        image_header,
        input_url,
        input_text,
        widgets.HBox(
            children=[
                button_toggle,
                button_clear,
                button_go,
                output_message,
            ],
        ),
        accordion_text,
        accordion_tree,
        accordion_graph,
        accordion_map,
        accordion_table,
    ],
    layout={
        'width': '90%',
        'left': '10%',
        'padding': '5px',
#         'border': '1px solid black',
    },
)
clear(None)
display(VBox_main)